In [1]:
import torch
import matplotlib.pyplot as plt
import numpy as np 
import argparse
import pickle 
import os
from torch.autograd import Variable 
from torchvision import transforms 
from data_loader import build_vocab 
from model import EncoderCNN, DecoderRNN
from model import ResNet, ResidualBlock
from PIL import Image


embed_size = 256
hidden_size = 512 
num_layers = 1 
decoder_path = './models/1object_color/decoder-10-150.pkl'
encoder_path = './models/1object_color/encoder-10-150.pkl'


def to_var(x, volatile=False):
    if torch.cuda.is_available():
        x = x.cuda()
    return Variable(x, volatile=volatile)

def load_image(image_path, transform=None):
    image = Image.open(image_path).convert('RGB')
    image = image.resize([64, 64], Image.LANCZOS)
    
    if transform is not None:
        image = transform(image).unsqueeze(0)
    
    return image

transform = transforms.Compose([ 
    transforms.ToTensor(), 
    transforms.Normalize((0.033, 0.032, 0.033), 
                         (0.027, 0.027, 0.027))])



vocab_path = './data/vocab_1object_color.pkl'
with open(vocab_path, 'rb') as f:
    vocab = pickle.load(f)
len_vocab = vocab.idx

# Build Models
encoder = ResNet(ResidualBlock, [3, 3, 3], len_vocab)
encoder.eval()  # evaluation mode (BN uses moving mean/variance)
decoder = DecoderRNN(len_vocab, hidden_size, 
                     len(vocab), num_layers)


# Load the trained model parameters
encoder.load_state_dict(torch.load(encoder_path))
decoder.load_state_dict(torch.load(decoder_path))

# If use gpu
if torch.cuda.is_available():
    encoder.cuda()
    decoder.cuda()

    

In [2]:
test_dir = 'data/circle_and_rect/bitmap/'
cap_dir = 'data/circle_and_rect//caption/'
test_list = os.listdir(test_dir)
cnt = 0 

shape_match = 0
location_match = 0
radius_match = 0 
rect_x_match = 0 
rect_y_match = 0 
length_match = 0
circle_cnt = 0 
rect_cnt = 0 
color_match= 0

location_err = [] 
rect_x_err = []
rect_y_err = []
radius_err = [] 

for fname in test_list: 
    #cnt+=1 
    if cnt>2:
        break;
    test_path = test_dir + fname
    test_image = load_image(test_path, transform)
    image_tensor = to_var(test_image)
    
    # Generate caption from image
    feature = encoder(image_tensor)
    sampled_ids = decoder.sample(feature)
    ids_arr = []
    for element in sampled_ids: 
        temp = element.cpu().data.numpy()
        ids_arr.append(int(temp))
    #print(fname)

    # Decode word_ids to words
    sampled_caption = []
    for word_id in ids_arr:
        word = vocab.idx2word[word_id]
        sampled_caption.append(word)
        if word == '<end>':
            break
    in_caption = sampled_caption[1:-1]
    print(in_caption)
    
    #read target caption
    cap_target = cap_dir+fname
    cap_path = cap_target.split('.')[0]+'.svg'
    trg_caption = [] 
    with open(cap_path, 'r') as f:
        target = f.readline().split()
        for word in target:
            trg_caption.append(word)
    print(trg_caption)
    print('-------------------------------------------------------------------')
    
    if(trg_caption[0]) == 'circle':
        if in_caption[0] == trg_caption[0]:
            shape_match+=1
            circle_cnt+=1
        if (in_caption[1] == trg_caption[1]):
            location_match+=1
        if (in_caption[2] == trg_caption[2]):
            location_match+=1        
        if (in_caption[3] == trg_caption[3]):
            radius_match+=1
        else:
            error = int(trg_caption[3]) - int(in_caption[3])
            radius_err.append(abs(error))
        if (in_caption[4] == trg_caption[4]):
            color_match+=1
    elif(trg_caption[0]) == 'rect':
        if in_caption[0] == trg_caption[0]:
            shape_match+=1
            rect_cnt+=1
        if (in_caption[1] == trg_caption[1]):
            location_match+=1
        if (in_caption[2] == trg_caption[2]):
            location_match+=1        
        if (in_caption[3] == trg_caption[3]):
            rect_x_match+=1
        else: 
            error = int(trg_caption[3]) - int(in_caption[3])
            rect_x_err.append(abs(error))
        if (in_caption[4] == trg_caption[4]):
            rect_y_match+=1
        else: 
            error = int(trg_caption[4]) - int(in_caption[4])
            rect_y_err.append(abs(error))            
        if (in_caption[5] == trg_caption[5]):
            color_match+=1
            
    
    if(trg_caption[0] != in_caption[0]):
        print(fname)
    if(trg_caption[1] != in_caption[1]):
        try:
            error = int(trg_caption[1]) - int(in_caption[1])
        except:
            print(fname)
            print(in_caption)
            pass
        location_err.append(abs(error))
    if(trg_caption[2] != in_caption[2]):
        error = int(trg_caption[1]) - int(in_caption[2])
        location_err.append(abs(error))
            



['circle', '9', '5', '100', 'pink']
['circle', '9', '5', '100', 'deep_pink']
-------------------------------------------------------------------
['rect', '10', '10', '70', '70', 'lime']
['rect', '10', '10', '110', '110', 'orange']
-------------------------------------------------------------------
['circle', '10', '3', '100', 'pink']
['circle', '10', '3', '110', 'pink']
-------------------------------------------------------------------
['circle', '1', '9', '50', 'pink']
['circle', '1', '9', '60', 'deep_pink']
-------------------------------------------------------------------
['circle', '8', '5', '90', 'pink']
['circle', '8', '6', '100', 'purple']
-------------------------------------------------------------------
['circle', '6', '3', '40', 'pink']
['circle', '6', '3', '40', 'pink']
-------------------------------------------------------------------
['rect', '5', '5', '120', '120', 'orange']
['rect', '5', '5', '120', '120', 'red']
------------------------------------------------------

In [ ]:
trg_caption[0]
in_caption[0]

In [ ]:
in_caption[0]

In [ ]:
temp = [] 
temp.append('1')

In [20]:
print('shape match: %.4f, location match: %.4f, radius_match: %.4f, rect_x_match: %.4f, rect_y_match: %.4f color_match: %.4f'
     %(shape_match/(len(test_list)), location_match/(len(test_list)*2),
       radius_match/(circle_cnt), rect_x_match/rect_cnt, rect_y_match/rect_cnt, color_match/(len(test_list))))    

shape match: 0.9880, location match: 0.8892, radius_match: 0.5994, rect_x_match: 0.5732, rect_y_match: 0.5732 color_match: 0.0910


In [21]:
x_ = {x:rect_x_err.count(x) for x in rect_x_err}
y_ = {x:rect_y_err.count(x) for x in rect_y_err}
r_ = {x:radius_err.count(x) for x in radius_err}
loc_ ={x:location_err.count(x) for x in location_err}

In [26]:
loc_

{0: 25, 1: 244, 2: 51, 3: 42, 4: 22, 5: 15, 6: 13, 7: 17, 8: 10, 9: 3, 100: 1}

In [14]:
x_[10]/len(rect_x_err)

0.9033816425120773